In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
from utils import *
from models import *
from data import *

PATH = os.getcwd()
DATA_PATH = "{}/data".format(PATH)

torch.__version__

'0.4.0'

## Predict 1600x1600 crops, stitch them to 3200x3200 together and save them for submitting

In [2]:
def process_and_predict(img, model):
    img = img.unsqueeze(0)
#     pdb.set_trace()
    pred = model(img)[0,0]
    pred[pred>0.5] = 1
    pred[pred!=1] = 0
    return pred

debug = False
num_gpu = 1
name = "UNet_BN_crop80_lr0.1_noaugm_logloss0.1_dice_0.9_bs256_sample20k"
create_dir("subm")
create_dir("{}/{}".format(DATA_PATH, name))
class_list = ["Buildings", "Misc. Manmade structures", "Road", "Track", "Trees", "Crops", "Waterway",
              "Standing Water", "Vehicle Large", "Vehicle Small"]
SB = pd.read_csv('data/sample_submission.csv')

for class_nb in range(0, 10):
    count = 0
    create_dir("{}/{}/{}".format(DATA_PATH, name, class_list[class_nb]))
    hps = HyperParams()
    hps.update("net=UNet_BN,bn=1,classes={}".format(str(class_nb)))
    
    model = UNet_BN(hps)
    arg_name = "UNet_BN_{}_crop80_lr0.1_noaugm_logloss0.1_dice_0.9_bs256_sample20k".format(class_list[class_nb])
    checkpoint = torch.load("weights/{}/model_best.pth.tar".format(arg_name))
    print("Loading {} with {:.3f} Jaccard".format("weights/{}/model_best.pth.tar".format(arg_name), checkpoint["best_jaccard"]))
    model = torch.nn.DataParallel(model, device_ids=list(range(num_gpu))).cuda()
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    
    for idx, row in tqdm(SB.iterrows(), total=SB.shape[0]):
        id_ = row[0]
        kls = row[1] - 1
        if kls!=class_nb:
            continue
        with torch.no_grad():
            count += 1
#             if count > 20:
#                 break
#             start = time.time()
            img = M(id_, dims=12)
#             print("loading takes {:.1f}s".format(time.time()-start))
#             start = time.time()
            img = cv2.resize(img, (3200,3200))
            if debug:
                f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 20))
                ax1.imshow(stretch_8bit(img[:,:, :3]))
#             print("resizing takes {:.1f}s".format(time.time()-start))
#             start = time.time()
            h, w = img.shape[:2]
            
            img = torch.from_numpy(img.transpose((2, 0, 1)).copy())
            img = normalize(img)

            pred_mask = np.zeros((h, w), dtype=np.uint8)
            img_ = img[:, :int(0.5*h), :int(0.5*w)]
            pred_mask[:int(0.5*h), :int(0.5*w)] = process_and_predict(img_, model)
            img_ = img[:, :int(0.5*h), int(0.5*w):]
            pred_mask[:int(0.5*h), int(0.5*w):] = process_and_predict(img_, model)
            img_ = img[:, int(0.5*h):, :int(0.5*w)]
            pred_mask[int(0.5*h):, :int(0.5*w)] = process_and_predict(img_, model)
            img_ = img[:, int(0.5*h):, int(0.5*w):]
            pred_mask[int(0.5*h):, int(0.5*w):] = process_and_predict(img_, model)
            if debug:
                ax2.imshow(pred_mask*255, cmap="gray")
                plt.show()
#             print("predicting and stitching takes {:.1f}s".format(time.time()-start))
            save_array("{}/{}/{}/{}.bc".format(DATA_PATH, name, class_list[class_nb], id_),  pred_mask)